In [1]:
epochs = 10
n_train_items = 1280

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np

In [3]:
import syft as sy  # <-- NEW: import the Pysyft library
hook = sy.TorchHook(torch)  # <-- NEW: hook PyTorch ie add extra functionalities to support Federated Learning
# simulation functions
def connect_to_workers(n_workers):
    return [
        sy.VirtualWorker(hook, id=f"worker{i+1}")
        for i in range(n_workers)
    ]


workers = connect_to_workers(n_workers=20)


/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

In [4]:
class Arguments():
    def __init__(self):
        self.batch_size = 64
        self.test_batch_size = 64
        self.epochs = epochs
        self.lr = 0.01
        self.momentum = 0.5
        self.no_cuda = False
        self.seed = 1
        self.log_interval = 3
        self.save_model = False

args = Arguments()

use_cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)

device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

In [5]:
# federated_train_loader = sy.FederatedDataLoader( # <-- this is now a FederatedDataLoader 
#     datasets.MNIST('../data', train=True, download=True,
#                    transform=transforms.Compose([
#                        transforms.ToTensor(),
#                        transforms.Normalize((0.1307,), (0.3081,))
#                    ]))
#     .federate(workers), # <-- NEW: we distribute the dataset across all the workers, it's now a FederatedDataset
#     batch_size=args.batch_size, shuffle=True, **kwargs)

# test_loader = torch.utils.data.DataLoader(
#     datasets.MNIST('../data', train=False, transform=transforms.Compose([
#                        transforms.ToTensor(),
#                        transforms.Normalize((0.1307,), (0.3081,))
#                    ])),
#     batch_size=args.test_batch_size, shuffle=True, **kwargs)

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.batch_size
)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, download=True, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.test_batch_size
)

    
#---

less_train_dataloader = [
        ((data), (target))
        for i, (data, target) in enumerate(train_loader)
        if i < n_train_items / args.batch_size
    ]
less_test_dataloader = [
        ((data), (target))
        for i, (data, target) in enumerate(test_loader)
        if i < n_train_items / args.batch_size
    ]




In [6]:
# from PIL import Image
# import numpy 
# #mnist_dataset.__getitem__(2)[1]
# a = (mnist_dataset.__getitem__(0)[0]).numpy()
# a.dtype = 'uint8'
# print(a)
# Image.fromarray(a[0], mode= 'P')

In [7]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [17]:
def model_init(workers, Net):
    model_list = list()
    for worker in workers:
        model_list.append(Net)
    return model_list
def opt_init(model_list):
    opt_list = list()
    for model  in model_list:
        opt_list.append(optim.SGD(model.parameters(), lr=args.lr))
    return opt_list

In [111]:
def train(args, model_list, device, less_train_dataloader, opt_list, epoch, workers):
    
    ## start training and record the model into model_list
    
    for batch_idx, (data, target) in enumerate(less_train_dataloader): # <-- now it is a distributed dataset
        model_on_worker = model_list[batch_idx%len(workers)].copy()
        model_on_worker.train()
        model_on_worker.send(workers[batch_idx%len(workers)]) # <-- NEW: send the model to the right location
        
        data_on_worker = data.send(workers[batch_idx%len(workers)])
        target_on_worker = target.send(workers[batch_idx%len(workers)])
        
        data_on_worker, target_on_worker = data_on_worker.to(device), target_on_worker.to(device)
        
        opt_list[batch_idx%len(workers)].zero_grad()
        
        output = model_on_worker(data_on_worker)
        loss = F.nll_loss(output, target_on_worker)
        loss.backward()
        opt_list[batch_idx%len(workers)].step()
        model_on_worker.get() # <-- NEW: get the model back
        
        model_list[batch_idx%len(workers)] = model_on_worker #When len(dataloader) is longer than the len(worker) send and get must be modified
        #model_list here is full of the model which has trained on the workers, there are all different now.
        
        if batch_idx % args.log_interval == 0:
            loss = loss.get() # <-- NEW: get the loss back
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * args.batch_size, len(less_train_dataloader) * args.batch_size,
                100. * batch_idx / len(less_train_dataloader), loss.item()))


    ##Aggregation time
    new_model = []
    tmp_model = Net().to(device)
    with torch.no_grad():
        for p in model_list[0].parameters():
            new_model.append(0)
            
        for m in model_list:
            for par_idx, par in enumerate(m.parameters()):
                #average the model_list
                new_model[par_idx] = new_model[par_idx]+par.data
                # we get new model in list format and need to set_ to model
        
        for i, model in enumerate(model_list):
            for i, par in enumerate(tmp_model.parameters()):
            par.set_(new_model[i]/len(workers))
        # Get new model in tmp_model
                  
            
        #init model with new_model
        for workers in range(len(workers):
            for par in range(len(new_model)):
                model_list[workers][par].set_(tmp)



In [122]:
for i,j in enumerate(model.parameters()):
    print(i)

0
1
2
3
4
5
6
7


In [113]:
def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader*args.batch_size)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader* args.batch_size),
        100. * correct / (len(test_loader)*args.batch_size)))

In [119]:
%%time
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=args.lr) # TODO momentum is not supported at the moment

model_list = []
model_list = model_init(workers, Net().to(device))
opt_list = opt_init(model_list)
# not finish in train, finish latter

for epoch in range(1, args.epochs + 1):
    train(args, model_list, device, less_train_dataloader, opt_list, epoch, workers)
    print("After training")
    test(args, model_list[0], device, less_test_dataloader)

if (args.save_model):
    torch.save(model.state_dict(), "mnist_cnn.pt")

Train Epoch: 1 [0/1280 (0%)]	Loss: 2.315260
Train Epoch: 1 [192/1280 (15%)]	Loss: 2.314102
Train Epoch: 1 [384/1280 (30%)]	Loss: 2.307504
Train Epoch: 1 [576/1280 (45%)]	Loss: 2.313025
Train Epoch: 1 [768/1280 (60%)]	Loss: 2.298383
Train Epoch: 1 [960/1280 (75%)]	Loss: 2.304842
Train Epoch: 1 [1152/1280 (90%)]	Loss: 2.324359
After training

Test set: Average loss: 2.3144, Accuracy: 116/1280 (9%)

Train Epoch: 2 [0/1280 (0%)]	Loss: 2.315260
Train Epoch: 2 [192/1280 (15%)]	Loss: 2.314102
Train Epoch: 2 [384/1280 (30%)]	Loss: 2.307504
Train Epoch: 2 [576/1280 (45%)]	Loss: 2.313025
Train Epoch: 2 [768/1280 (60%)]	Loss: 2.298383
Train Epoch: 2 [960/1280 (75%)]	Loss: 2.304842
Train Epoch: 2 [1152/1280 (90%)]	Loss: 2.324359
After training

Test set: Average loss: 2.3144, Accuracy: 116/1280 (9%)

Train Epoch: 3 [0/1280 (0%)]	Loss: 2.315260
Train Epoch: 3 [192/1280 (15%)]	Loss: 2.314102
Train Epoch: 3 [384/1280 (30%)]	Loss: 2.307504
Train Epoch: 3 [576/1280 (45%)]	Loss: 2.313025
Train Epoch: 3 

In [97]:
model = Net()
tttt = sy.VirtualWorker(hook, id="tttt")
tttt.clear_objects()
model = model.send(tttt)


t = torch.ones([1,28,28], dtype=torch.float64)
t_on_worker = t.send(tttt)
print((t_on_worker))
model(t)
# print(workers[0].current_objects())


(Wrapper)>[PointerTensor | me:52758697780 -> tttt:10731448168]


RuntimeError: weight should have at least three dimensions

In [13]:
model = Net()
model_list = []
model_ondevice = []
aggregater = sy.VirtualWorker(hook, id="aggregater")

for batch_idx, (data, target) in enumerate(less_train_dataloader):
    data_on_device = data.send(workers[batch_idx])
    if batch_idx<3:
        model_ondevice.append(model.copy().send(workers[batch_idx]))
        print(model_ondevice[batch_idx].location)
        
        pre = model_ondevice[batch_idx](data_on_device)
        model_ondevice[batch_idx].move(aggregater)
        print(model.location)
        print(model.weight.data)


<VirtualWorker id:worker1 #objects:15>
None


AttributeError: 'Net' object has no attribute 'weight'

In [ ]:
with torch.no_grad():
    print(model_list[0].parameters())

In [250]:
model.parameters()
model.sned(workers[0])


<generator object Module.parameters at 0x7fb2982e66d0>

In [296]:
for i in model.parameters():
    test.append(i.data)
new_test = []
for i in test:
    new_test.append(i+i)
tmp_model = Net()
with torch.no_grad():
    for i,par in enumerate(tmp_model.parameters()):
        par.set_(new_test[i])
for i in tmp_model.parameters():
    print(i)

Parameter containing:
tensor([[[[-2.8198e-01, -2.1895e-02, -2.8959e-01, -2.0107e-01,  2.2971e-01],
          [ 1.8315e-01,  7.7493e-02,  1.6640e-01,  2.1500e-02,  1.6269e-01],
          [ 3.7304e-01, -3.2290e-01, -1.3509e-01,  2.8803e-01,  2.2667e-01],
          [ 3.1194e-01, -3.9431e-01,  1.6057e-01,  3.3954e-01,  3.6352e-01],
          [-3.4737e-01,  2.0403e-01,  2.6506e-01,  1.0429e-01,  2.8679e-01]]],


        [[[-6.6582e-02, -2.7106e-01,  1.8387e-01, -3.1548e-01,  5.3829e-02],
          [-2.3096e-02,  2.5281e-01, -3.1633e-01,  3.5198e-01,  4.2042e-02],
          [-2.1132e-01, -2.4991e-01,  3.6416e-01, -3.2156e-01, -1.2817e-01],
          [-2.7399e-01,  3.2240e-01, -2.3954e-01, -1.1032e-01,  4.2613e-02],
          [-2.7170e-01, -3.2338e-01, -3.1872e-01,  3.4642e-01,  1.2469e-01]]],


        [[[-4.1491e-02,  3.6591e-01, -1.2837e-01,  3.6525e-01,  2.7922e-01],
          [-9.6500e-02,  2.7981e-01, -1.3758e-01, -1.5327e-01,  4.9002e-02],
          [ 1.0895e-01,  1.1725e-01,  3.1835e-

In [283]:
x = list(torch.tensor((1,2)))
for i in x:
    i+i

In [231]:
model.fc2.weight.data.numpy()

array([[-0.01297603,  0.02842132,  0.01014335, ..., -0.00745563,
        -0.02407177, -0.00225234],
       [ 0.03080916,  0.01323886,  0.03059926, ..., -0.01799139,
        -0.0265331 ,  0.01559756],
       [ 0.01701068,  0.00417169,  0.0316051 , ...,  0.03077709,
        -0.02863705,  0.01833366],
       ...,
       [-0.00771326, -0.01363829,  0.03800831, ..., -0.04430421,
        -0.03094019, -0.01628821],
       [-0.0320928 ,  0.03452658,  0.01318498, ..., -0.0185255 ,
         0.04329454,  0.01331757],
       [ 0.02591962, -0.00609975,  0.00413742, ...,  0.02711744,
        -0.00503955, -0.03584282]], dtype=float32)

In [233]:
model.fc2.bias.set_

<function Parameter.set_>